<a href="https://colab.research.google.com/github/aswinaus/RAG/blob/main/RAG_DeepSeekR1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install git+https://github.com/huggingface/transformers torch accelerate bitsandbytes langchain

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-enn6s1l4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-enn6s1l4
  Resolved https://github.com/huggingface/transformers to commit 7547f55e5d93245c0a013b50df976924f2d9e8b0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.4 MB/s

Code is essentially forcing Python to always use "UTF-8" as the preferred encoding, regardless of the user's actual system settings. UTF-8 is a widely used encoding that can represent a vast range of characters from different languages. By enforcing UTF-8, you can help ensure that your code works consistently across different platforms and avoids encoding-related errors. It's a common practice for improving compatibility and preventing issues with text handling in Python programs.

In [3]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
#pyngrok is a Python wrapper for ngrok, a tool that allows you to expose a local web server to the public internet. This can be very useful for sharing your work, testing webhooks, or building demos that need to be accessible from the outside.
!pip install pyngrok --quiet
#This is a modern, fast (high-performance) web framework for building APIs with Python 3.6+ based on standard Python type hints. It's often used for creating web applications and services.
!pip install fastapi nest-asyncio --quiet
#uvicorn is an ASGI (Asynchronous Server Gateway Interface) web server. This essentially means that it's a tool that can run Python web applications designed for asynchronous operation and handling many requests concurrently. It's often used with modern Python web frameworks like FastAPI to serve the application to users.
!pip install uvicorn --quiet
!pip install langchain-community --quiet
!pip install -U langchain-huggingface --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 4.8 MB/s eta 0:00:00


In [5]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [6]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Agents` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Agents`


In [7]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

device = 'cuda' if torch.cuda.is_available() else 'cpu'

The nvidia-smi command is a utility provided by NVIDIA to query and display information about your NVIDIA GPU(s) (Graphics Processing Unit). This includes things like:

GPU model and name
Driver version
GPU utilization
Memory usage
Temperature
Power consumption
Processes running on the GPU

In [8]:
!nvidia-smi

Wed Jan 29 23:53:09 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             44W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [9]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [10]:
from typing import Tuple, Optional, Union, Dict, Any
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, AutoConfig
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

Function provides a robust way to load a pre-trained model, prioritizing quantization for optimization but gracefully falling back to a non-quantized version if necessary. This helps ensure compatibility and flexibility when working with different models and environments

In [11]:
def load_model_with_quantization_fallback(
    model_name: str = "deepseek-ai/deepseek-llm-7b-chat",
    trust_remote_code: bool = True,
    **kwargs
) -> Tuple[PreTrainedModel, PreTrainedTokenizerBase]:

  try:
      model = AutoModel.from_pretrained(
          model_name,
          trust_remote_code=trust_remote_code,
          device_map=device_map,
          **kwargs
      )
      tokenizer = AutoTokenizer.from_pretrained(model_name)
      print("Model loaded successfully with original configuration")
      return model, tokenizer
  except ValueError as e:
      if "Unknown quantization type" in str(e):
          print(
              "Quantization type not supported directly. "
              "Attempting to load without quantization..."
          )

          config = AutoConfig.from_pretrained(
              model_name,
              trust_remote_code=trust_remote_code
          )
          if hasattr(config, "quantization_config"):
              delattr(config, "quantization_config")

          try:
              model = AutoModel.from_pretrained(
                  model_name,
                  config=config,
                  trust_remote_code=trust_remote_code,
                  device_map=device_map,
                  **kwargs
              )
              tokenizer = AutoTokenizer.from_pretrained(
                  model_name,
                  trust_remote_code=trust_remote_code
              )
              print("Model loaded successfully without quantization")
              return model, tokenizer

          except Exception as inner_e:
              print(f"Failed to load model without quantization: {str(inner_e)}")
              raise
      else:
          print(f"Unexpected error during model loading: {str(e)}")
          raise

In [12]:
import torch

# Determine the device to load the model on (CPU or GPU)
device_map = "cuda" if torch.cuda.is_available() else "cpu"

# Now you can call the function:
model, tokenizer = load_model_with_quantization_fallback()

config.json:   0%|          | 0.00/594 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/22.5k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.6k [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.85G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.61M [00:00<?, ?B/s]

Model loaded successfully with original configuration


In [13]:
!pip install datasets langchain langchain_community langchain_openai chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.2 MB/s

In [14]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_openai import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain_core.runnables import (
    RunnableParallel,
    RunnablePassthrough
)
from langchain.schema.output_parser import StrOutputParser

In [15]:
from getpass import getpass
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [16]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("aswinaus/tax_statistics_dataset_by_income_range", download_mode="force_redownload")
df=pd.DataFrame(dataset['train'])

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

(…)x_statistics_dataset_by_income_range.csv:   0%|          | 0.00/272k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/303 [00:00<?, ? examples/s]

In [17]:
from google.colab import drive
drive.mount('/content/drive')
# Download Data
data_dir = '/content/drive/MyDrive'

Mounted at /content/drive


In [18]:
from typing import Dict, Any, List
from langchain.docstore.document import Document

In [19]:
# Convert DatasetDict to LangChain Documents
def create_langchain_documents(dataset: Dict[str, Any]) -> List[Document]:
    """Converts a Hugging Face DatasetDict to a list of LangChain Documents,
    including all columns as content.
    """
    documents = []
    for row in dataset['train']:  # Assuming 'train' is your split name
        # Concatenate all column values into a single string
        content = "\n".join([f"{k}: {v}" for k, v in row.items()])

        # Use all columns except 'content' as metadata
        metadata = row.copy()

        document = Document(page_content=content, metadata=metadata)
        documents.append(document)
    return documents

In [20]:
all_documents = create_langchain_documents(dataset)
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50,
)

In [21]:
pages = []
for document in all_documents:
  pages.extend(text_splitter.split_documents([document]))

In [22]:
# create vector store with Chroma
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores.utils import filter_complex_metadata # import filter_complex_metadata

vectordb = Chroma.from_documents(documents=pages, embedding=OpenAIEmbeddings(openai_api_key=os.environ["OPENAI_API_KEY"]),persist_directory=f"{data_dir}/RAG/VectorDB/chroma_db_RAG_Income_Tax")
vectordb.persist()
retriever = vectordb.as_retriever()

<ipython-input-22-a88b0c424d3f>:6: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [23]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [24]:
def encode_prompt(inputs: Dict[str, Any]) -> Dict[str, Any]:
    input_text = inputs["prompt"]
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    print(f"Input text: {input_text}")
    print(f"Input IDs: {input_ids}")
    # Instead of embedding here, return the input_ids directly
    #inputs["prompt"] = input_ids  # Replace text with tensor
    return input_ids # Return the tensor directly

In [25]:
!pip install langchain_huggingface --quiet

In [26]:
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import transformers

In [27]:
from langchain_core.runnables import RunnableSequence
# RAG
template = """You are an AI language model Accounting assistant.Answer the following question based on this context:
{context}
Question: {question}
"""
prompt_creator = ChatPromptTemplate.from_template(template) # moved template to prompt_creator


In [28]:
#Creating a RAG Pipeline
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers import StrOutputParser

# RAG
template = """You are an AI language model Accounting assistant.Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = prompt_creator
llm=model
final_rag_chain=RunnableSequence(
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
    )
    |{"context": retriever | format_docs, "question": RunnablePassthrough()}
    | RunnableMap({"prompt": encode_prompt})
    | model
    | StrOutputParser()
)

#prompt = ChatPromptTemplate.from_template(template)
#llm = model#ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])



In [29]:
#Creating a RAG Pipeline
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableMap
from langchain_core.output_parsers import StrOutputParser
from langchain.schema import HumanMessage
#from langchain.chains import RunnableSequence # Add this import

# RAG
template = """You are an AI language model Accounting assistant.Answer the following question based on this context:
{context}
Question: {question}
"""

prompt = prompt_creator
llm = model#ChatOpenAI(temperature=0, openai_api_key=os.environ["OPENAI_API_KEY"])
final_rag_chain = RunnableSequence(
    RunnablePassthrough.assign(
        context=lambda x: format_docs(retriever.get_relevant_documents(x["question"])),
    )
    | RunnablePassthrough.assign(
        prompt=lambda x: prompt_creator.format(context=x["context"], question=x["question"]) # format template string
    ) # This was missing before RunnableMap, leading to KeyError: 'prompt'

    | RunnablePassthrough.assign(debug_context=lambda x: print(f"Context before prompt: {x['context']}"))
    | RunnablePassthrough.assign(debug_question=lambda x: print(f"Question before prompt: {x['question']}"))

    | RunnablePassthrough.assign(debug_prompt=lambda x: print(f"Prompt after prompt: {x['prompt']}"))
    # Modified to input the prompt to encode_prompt and extract the tensor
    | RunnableMap({"prompt": lambda x: encode_prompt({"prompt": x['prompt']})})
    # Extract the tensor from the dictionary and assign to 'input_ids'
    | RunnablePassthrough.assign(input_ids=lambda x: x["prompt"])
    # Call the model with the correct arguments (input_ids)
    | (lambda x: llm(input_ids=x['input_ids']))
    # Decode the output
    | (lambda x: tokenizer.decode(x.logits[0], skip_special_tokens=True))
    | StrOutputParser()
    #Modified line
    #| (lambda x: llm.generate(input_ids=x['prompt'], max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id))
    #Modified line
    #| (lambda x: tokenizer.decode(x[0], skip_special_tokens=True))
)



In [31]:
question="What is the number of joint returns for the state of AL for income range $100,000 under $200,000?"


In [32]:
final_rag_chain.invoke({"question":question})
#final_rag_chain.invoke(question)

<ipython-input-29-59c9fa64d86f>:20: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context=lambda x: format_docs(retriever.get_relevant_documents(x["question"])),


Context before prompt: STATEFIPS: 1
STATE: AL
zipcode: 0
Size of adjusted gross income: $100,000 under $200,000
No of returns: 257010
No of single returns: 28180
No of joint returns: 216740
No of head of household returns: 7150
Number of electronically filed returns: 236850
Number of computer prepared paper returns: 8400
Number of returns with paid preparer's signature: 149420
Number of returns with direct deposit: 134040
Number of individuals: 692450
Total number of volunteer prepared returns: 420
Number of volunteer income tax assistance VITA prepared returns: 420
Number of tax counseling for the elderly  TCE prepared returns: 0
Number of volunteer prepared returns with Earned Income Credit: 0
Number of refund anticipation check returns: 16120
Number of elderly returns: 85030
Adjust gross income AGI: 34974856
Number of returns with total income: 257010
Total income amount: 35286228
Number of returns with salaries and wages: 224330
Salaries and wages amount: 25607154
Number of returns

OutOfMemoryError: CUDA out of memory. Tried to allocate 28.00 MiB. GPU 0 has a total capacity of 39.56 GiB of which 20.88 MiB is free. Process 10850 has 39.53 GiB memory in use. Of the allocated memory 38.43 GiB is allocated by PyTorch, and 626.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)